## Model Training

### Notes
#### Predictions will be based on
- Smoker
- Smoker * BMI
- Region
- Gender, Maybe
- Age


#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

#### Import the CSV Data as Pandas DataFrame

In [3]:
##Reads the data set from csv
df = pd.read_csv('data/insurance.csv')

##Removes duplicates
df = df.drop_duplicates()

##Add the Feature Engineering
df['smoker_binary'] = df['smoker'].map({'yes': 1, 'no': 0})
df['bmi_smoker_interaction'] = df['bmi'] * df['smoker_binary']

## displays top 5 entries
df.head()


,age,sex,bmi,children,smoker,region,charges,smoker_binary,bmi_smoker_interaction
0,19,female,27.900,0,yes,southwest,16884.92400,1,27.9
1,18,male,33.770,1,no,southeast,1725.55230,0,0.0
2,28,male,33.000,3,no,southeast,4449.46200,0,0.0
3,33,male,22.705,0,no,northwest,21984.47061,0,0.0
4,32,male,28.880,0,no,northwest,3866.85520,0,0.0


#### Preparing X and Y variables

In [4]:
X = df.drop(columns=['bmi', 'children', 'smoker', 'charges'],axis=1)

In [5]:
X.head()

,age,sex,region,smoker_binary,bmi_smoker_interaction
0,19,female,southwest,1,27.9
1,18,male,southeast,0,0.0
2,28,male,southeast,0,0.0
3,33,male,northwest,0,0.0
4,32,male,northwest,0,0.0


In [6]:
print("Categories in 'sex' variable:     ",end=" " )
print(df['sex'].unique())

print("Categories in 'region' variable:     ",end=" " )
print(df['region'].unique())

print("Categories in 'smoker' variable:     ",end=" " )
print(df['smoker'].unique())

Categories in 'sex' variable:      ['female' 'male']
Categories in 'region' variable:      ['southwest' 'southeast' 'northwest' 'northeast']
Categories in 'smoker' variable:      ['yes' 'no']


In [7]:
y = df['charges']

In [8]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1337, dtype: float64

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Identify numerical and categorical features
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

# Z Score Normalization for numerical features
numeric_transformer = StandardScaler()

# OneHotEncoder for categorical features
oh_transformer = OneHotEncoder()

# ColumnTransformer with separate treatment for smoker_binary
preprocessor = ColumnTransformer(
    transformers=[
        ("OneHotEncoder", oh_transformer, cat_features),  # Apply OneHotEncoder to categorical features
        ("StandardScaler", numeric_transformer, num_features.difference(['smoker_binary'])),  # Scale all numerical features except smoker_binary
        ('passthrough', 'passthrough', ['smoker_binary'])  # Leave smoker_binary unchanged
    ]
)

In [10]:
X = preprocessor.fit_transform(X)

In [11]:
X.shape

(1337, 9)

In [12]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

print(X_train.shape)

(1069, 9)


#### Create an Evaluate Function to give all metrics after model Training

In [13]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 4975.5614
- Mean Absolute Error: 3010.0046
- R2 Score: 0.8192
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 4569.3102
- Mean Absolute Error: 2812.3314
- R2 Score: 0.8864


Linear Regression formula:
Intercept: 2.123583684565473e+17
Coefficients: [ 9.79680324e+15  9.79680324e+15 -2.22155172e+17 -2.22155172e+17
 -2.22155172e+17 -2.22155172e+17  3.63956250e+03  1.90387500e+04
 -2.19467500e+04]
Equation: y = 212358368456547296.0000 + (9796803241048424.0000)*X1

 + (9796803241047562.0000)*X2

 + (-222155171697576160.0000)*X3

 + (-222155171697577440.0000)*X4

 + (-222155171697578528.0000)*X5

 + (-222155171697577728.0000)*X6

 + (3639.5625)*X7

 + (19038.7500)*X8

 + (-21946.7500)*X9

Lasso
Model performance for Training set
- Root Mean Squared Error: 4946.0635
- Mean Absolute Error: 3012.1710
- R2 Score: 0.8213
----------------------------------
Model performance f

### Results

In [15]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,Linear Regression,0.886379
2,Ridge,0.886331
1,Lasso,0.885809
7,CatBoosting Regressor,0.879552
5,Random Forest Regressor,0.843050
3,K-Neighbors Regressor,0.817836
6,XGBRegressor,0.814782
8,AdaBoost Regressor,0.792711
4,Decision Tree,0.791986


## Conclusion

- **Linear Regression**: Best performance with R² = **0.886379**.
- **Ridge**: Close to Linear Regression with R² = **0.886331**.
- **Lasso**: Similar to Ridge with R² = **0.885809**.
- **CatBoosting Regressor**: R² = **0.879552**, 
- **Random Forest Regressor**: R² = **0.843050**, 
- **K-Neighbors Regressor**: R² = **0.817836**.
- **XGBRegressor**: R² = **0.814782**.
- **AdaBoost Regressor**: R² = **0.792711**.
- **Decision Tree**: R² = **0.791986**, weakest performance.

Overall, **Linear Regression** outperforms other models.